For training/testing/evaluating the model

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch

# Load Sentiment model and tokenizer
sentiment_model_name = "cardiffnlp/twitter-roberta-base-sentiment"
sentiment_tokenizer = AutoTokenizer.from_pretrained(sentiment_model_name)
sentiment_model = AutoModelForSequenceClassification.from_pretrained(sentiment_model_name)

# Load Emotion model and tokenizer
emotion_model_name = "j-hartmann/emotion-english-distilroberta-base"
emotion_tokenizer = AutoTokenizer.from_pretrained(emotion_model_name)
emotion_model = AutoModelForSequenceClassification.from_pretrained(emotion_model_name)

/Users/aditya/Developer/fact_checker/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch.nn.functional as F

# Map for sentiment labels (cardiffnlp model)
sentiment_labels = ['Negative', 'Neutral', 'Positive']

# Map for emotion labels (j-hartmann model)
emotion_labels = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']

def predict_sentiment(text):
    inputs = sentiment_tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        outputs = sentiment_model(**inputs)
    scores = F.softmax(outputs.logits, dim=1)[0]
    max_score_idx = torch.argmax(scores).item()
    return {
        "label": sentiment_labels[max_score_idx],
        "score": scores[max_score_idx].item()
    }

def predict_emotion(text):
    inputs = emotion_tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        outputs = emotion_model(**inputs)
    scores = F.softmax(outputs.logits, dim=1)[0]
    max_score_idx = torch.argmax(scores).item()
    return {
        "label": emotion_labels[max_score_idx],
        "score": scores[max_score_idx].item()
    }


In [3]:
headline = "Breaking: Major Earthquake Strikes City"
article = """
A powerful earthquake measuring 7.8 magnitude struck the city early this morning,
causing widespread damage and panic among residents. Emergency services are responding.
"""

full_text = headline + " " + article

sentiment_result = predict_sentiment(full_text)
emotion_result = predict_emotion(full_text)

print("Sentiment:", sentiment_result)
print("Emotion:", emotion_result)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Sentiment: {'label': 'Negative', 'score': 0.7228479385375977}
Emotion: {'label': 'fear', 'score': 0.9862047433853149}
